## Create Synthetic Data for Grocery Supply Chain

Features of the adjusted data:
Specific categories: Using only the categories present in your data

Realistic distribution: Based on observed frequency in the provided data

Realistic parameters per category:

🥦 **Produce**
- **Lead Time:** 1–3 days (locally sourced), 5–10 days (imported)
- **Shelf Life:** 3–10 days (most fresh items), up to 2 weeks for hardy vegetables like carrots or potatoes

🌾 **Grains and Flours**
- **Lead Time:** 3–7 days (domestic), 10–15 days (imported specialty grains)
- **Shelf Life:** 6 months to 1 year (dry, sealed), up to 2 years for rice and flour stored properly

🧀 **Dairy and Cold Cuts**
- **Lead Time:** 2–5 days (regional suppliers), 7–10 days (specialty cheeses)
- **Shelf Life:**
  - Milk & cream: 7–14 days refrigerated
  - Yogurt & soft cheeses: 2–3 weeks
  - Hard cheeses: 1–3 months
  - Cold cuts: 1–2 weeks sealed

☕ **Beverages**
- **Lead Time:** 2–7 days (coffee/tea distributors)
- **Shelf Life:**
  - Tea: 1–2 years (dry)
  - Coffee beans: 6–12 months (sealed), 1–2 weeks after grinding
  - Brewed drinks: 1–3 days refrigerated

🥚 **Eggs and Poultry**
- **Lead Time:** 1–3 days (local farms), 5–7 days (wholesale)
- **Shelf Life:**
  - Eggs: 3–5 weeks refrigerated
  - Fresh poultry: 1–2 days raw, 3–4 days cooked

🐟 **Meats and Fish**
- **Lead Time:** 1–5 days (fresh), 7–10 days (frozen or imported)
- **Shelf Life:**
  - Fresh fish: 1–2 days
  - Frozen fish: 3–6 months
  - Cured fish (e.g., sardines): up to 1 year

🛢️ **Oils and Fats**
- **Lead Time:** 3–7 days (bulk suppliers)
- **Shelf Life:**
  - Vegetable oils: 6–12 months
  - Butter: 1 month refrigerated, 6 months frozen
  - Coconut oil: up to 2 years

🍬 **Sugars and Sweets**
- **Lead Time:** 2–5 days
- **Shelf Life:**
  - Sugars: indefinite if dry and sealed
  - Dried fruits (e.g., plum): 6–12 months

🍪 **Miscellaneous and Biscuits**
- **Lead Time:** 2–6 days
- **Shelf Life:**
  - Biscuits: 3–6 months sealed


Seasonal patterns:

- Fruits/vegetables with reduced shelf life in summer

- Dairy with shorter lead time in winter

Realistic temporal distribution:

- 80% of deliveries on weekdays

Controlled outliers: Only 3% of data with unusual situations

These synthetic data preserve the specific characteristics of the categories in your original dataset, with realistic temporal relationships for supply chain analysis.

## Data Generation
### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import json

from smart_supply_chain_ai.utils import create_data_functions, combine_df_functions, weather_conditions

import warnings
warnings.filterwarnings('ignore')

### Paths

In [2]:
# Define data paths
raw_data_path = os.path.join('../data', 'raw/')

external_data_path = os.path.join('../data', 'external/')

json_path = os.path.join('../src','smart_supply_chain_ai' , 'utils/')

In [3]:
# List of JSON filenames (without extension) to be loaded
arch_json = ['products','products_categories', 'suppliers']

# Dictionary to store the loaded JSON content
store_catalog = {}

# Loop through each filename, build the full path, and load the JSON data
for name in arch_json:
    file_path = os.path.join(json_path, f"{name}.json")  # Construct full file path
    with open(file_path, "r", encoding="utf-8") as f:     # Open the JSON file
        store_catalog[name] = json.load(f)                        # Load and store the data under its name

# Product catalog information

In [4]:
# Create a DataFrame of products with product names as a column
products = pd.DataFrame.from_dict(store_catalog['products']).T.reset_index().rename(columns={'index': 'product'})


In [5]:
# Replace product with new IDs
products['product_id'] = create_data_functions.create_IDs(products.shape[0], suffix='P')

# Supplier catalog and distribution details

In [6]:
# Create a DataFrame of suppliers with supplier names as a column
suppliers = pd.DataFrame.from_dict(store_catalog['suppliers']).T.reset_index().rename(columns={'index': 'supplier'})

In [7]:
# Insert supplier IDs as the second column
suppliers.insert(1, 'supplier_id', create_data_functions.create_IDs(suppliers.shape[0], suffix='S'))

In [8]:
# Remove 'category' and 'subcategories' columns from the suppliers DataFrame
suppliers.drop(columns=['category', 'subcategories'], inplace=True)


In [9]:
# Split each supplier's product list into separate rows and reset the index
suppliers = suppliers.explode('products').reset_index(drop=True)


In [10]:
# Merge product and supplier data on matching product names, then drop duplicate 'products' column from suppliers
products_suppliers_df = pd.merge(products, suppliers, left_on='product', right_on='products').drop(columns='products')

In [11]:
# Initialize a random number generator with a fixed seed for reproducibility.
rng = np.random.default_rng(seed=43)

# Assign random supplier ratings between 1 and 4 to all suppliers.
products_suppliers_df['supplier_rating'] = rng.integers(1, 5, size=products_suppliers_df.shape[0])

# Randomly select 15 unique suppliers to be considered "top suppliers".
suppliers_top = rng.choice(products_suppliers_df['supplier'].unique(), 15, replace=False)

# Update ratings: if the supplier is in the top list, set rating to 5; otherwise keep the original rating.
products_suppliers_df['supplier_rating'] = np.where(products_suppliers_df['supplier'].isin(suppliers_top), 5, products_suppliers_df['supplier_rating'])


## Meteorological Data for Supply Chain Management

In [12]:
# Set the path to the weather CSV file
# archive_csv = external_data_path + 'dados_83967_D_2015-01-01_2025-09-18.csv'
archive_csv = external_data_path + 'dados_B807_D_2022-12-07_2025-09-22.csv'

# Read the CSV file into a DataFrame
weather_df = pd.read_csv(archive_csv, sep=";", decimal=",", skiprows=9, engine="python")

# Show the first rows of the DataFrame
weather_df.head()

,Data Medicao,"PRECIPITACAO TOTAL, DIARIO (AUT)(mm)","TEMPERATURA MAXIMA, DIARIA (AUT)(°C)","TEMPERATURA MINIMA, DIARIA (AUT)(°C)","VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/s)",Unnamed: 5
0,2022-12-07,NaN,NaN,NaN,NaN,NaN
1,2022-12-08,NaN,NaN,NaN,NaN,NaN
2,2022-12-09,NaN,32.3,17.4,2.4,NaN
3,2022-12-10,0.0,30.1,18.3,2.2,NaN
4,2022-12-11,0.0,31.8,22.7,2.8,NaN


In [13]:
# Remove columns that contain only missing values
weather_df.dropna(axis=1, how='all', inplace=True)


In [14]:
# Rename columns to clear and descriptive English names
weather_df.columns = [
    "measurement_date",
    "daily_total_precipitation_mm",
    "daily_maximum_temperature_c",
    "daily_minimum_temperature_c",
    "daily_average_wind_speed_mps"
]


In [15]:
# Set 'measurement_date' as index and remove rows with all missing values
weather_df = weather_df.set_index('measurement_date').dropna(how='all')

# Show the first rows of the DataFrame
weather_df.head()


,daily_total_precipitation_mm,daily_maximum_temperature_c,daily_minimum_temperature_c,daily_average_wind_speed_mps
measurement_date,,,,
2022-12-09,NaN,32.3,17.4,2.4
2022-12-10,0.0,30.1,18.3,2.2
2022-12-11,0.0,31.8,22.7,2.8
2022-12-12,0.0,37.3,19.9,2.9
2022-12-13,6.8,25.3,17.6,3.7


In [16]:
# This line calculates the total number of missing (NaN) values in each column of the weather_df DataFrame.
weather_df.isna().sum()

daily_total_precipitation_mm    5
daily_maximum_temperature_c     2
daily_minimum_temperature_c     0
daily_average_wind_speed_mps    6
dtype: int64

In [17]:
# Fill missing values in 'daily_total_precipitation_mm' using backward fill (next valid value).
weather_df['daily_total_precipitation_mm'].fillna(method='bfill', inplace=True)

# First, fill missing values in 'daily_maximum_temperature_c' using backward fill.
weather_df['daily_maximum_temperature_c'].fillna(method='bfill', inplace=True)

# Then, fill any remaining missing values in 'daily_maximum_temperature_c' using forward fill.
weather_df['daily_maximum_temperature_c'].fillna(method='ffill', inplace=True)

# Fill missing values in 'daily_average_wind_speed_mps' using forward fill (previous valid value).
weather_df['daily_average_wind_speed_mps'].fillna(method='ffill', inplace=True)


In [18]:
# Reset the DataFrame index to a default integer index and drop the old index column.
weather_df = weather_df.reset_index()

# Convert 'measurement_date' column to datetime format
weather_df['measurement_date'] = pd.to_datetime(weather_df['measurement_date'])

In [19]:
# Display summary information about the DataFrame
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   measurement_date              992 non-null    datetime64[ns]
 1   daily_total_precipitation_mm  992 non-null    float64       
 2   daily_maximum_temperature_c   992 non-null    float64       
 3   daily_minimum_temperature_c   992 non-null    float64       
 4   daily_average_wind_speed_mps  992 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 38.9 KB


In [20]:
# Dropped all missing values
weather_df.dropna(inplace=True)

In [21]:
weather_df['measurement_date'].min()

Timestamp('2022-12-09 00:00:00')

In [22]:
# Apply the weather classification function to the cleaned DataFrame to generate severity and category labels
weather_analyser = weather_conditions.WeatherConditions(weather_df)
weather_severity_df = weather_analyser.classify_weather()

In [23]:
# Show 10 samples rows of the DataFrame
weather_severity_df.sample(10)

,measurement_date,daily_total_precipitation_mm,daily_maximum_temperature_c,daily_minimum_temperature_c,daily_average_wind_speed_mps,daily_average_temperature_c,temperature_classification,precipitation_classification,wind_classification,weather_severity
922,2025-07-15,1.6,16.5,12.1,1.0,14.30,Cool,Light Rain,Calm / Light Breeze,Moderate
234,2023-07-31,0.0,23.0,9.3,1.2,16.15,Cool,No precipitation,Calm / Light Breeze,Normal
749,2024-12-27,0.0,29.6,16.5,2.3,23.05,Mild to Temperate,No precipitation,Gentle to Fresh Breeze,Moderate
141,2023-04-29,0.0,25.8,10.6,1.6,18.20,Mild to Temperate,No precipitation,Gentle to Fresh Breeze,Moderate
954,2025-08-16,0.2,21.1,11.1,1.9,16.10,Cool,Light Rain,Gentle to Fresh Breeze,Moderate
838,2025-03-26,0.0,28.7,19.1,3.2,23.90,Mild to Temperate,No precipitation,Gentle to Fresh Breeze,Moderate
436,2024-02-18,0.0,30.6,20.3,3.0,25.45,Warm,No precipitation,Gentle to Fresh Breeze,Moderate
811,2025-02-27,1.8,30.6,23.5,2.3,27.05,Warm,Light Rain,Gentle to Fresh Breeze,Moderate
341,2023-11-15,0.0,24.4,20.1,1.3,22.25,Mild to Temperate,No precipitation,Calm / Light Breeze,Normal
599,2024-07-30,0.0,18.5,9.4,3.3,13.95,Cool,No precipitation,Gentle to Fresh Breeze,Moderate


In [24]:
# Generate and transpose summary statistics for all numeric columns in the classified weather DataFrame
weather_severity_df.describe().T

,count,mean,min,25%,50%,75%,max,std
measurement_date,992,2024-04-20 02:35:19.354838528,2022-12-09 00:00:00,2023-08-13 18:00:00,2024-04-17 12:00:00,2024-12-21 06:00:00,2025-09-22 00:00:00,NaN
daily_total_precipitation_mm,992.0,4.645565,0.0,0.0,0.0,1.65,132.0,12.602568
daily_maximum_temperature_c,992.0,25.034073,10.4,21.175,25.5,29.125,39.2,5.654455
daily_minimum_temperature_c,992.0,15.84496,1.7,12.3,16.6,19.6,25.2,4.928274
daily_average_wind_speed_mps,992.0,2.366331,0.7,1.6,2.3,2.9,7.7,0.96995
daily_average_temperature_c,992.0,20.439516,6.65,16.9375,20.85,24.25,30.9,4.911554


# Realistic supply chain modeling based on weather and product data

In [25]:
# Create a copy of the climate data DataFrame to work with weather-specific analysis
df_weather_conditions = weather_severity_df.copy()

# Create a copy of the products_suppliers data DataFrame to work with product-related operations
df_products = products_suppliers_df.copy()

In [26]:
# Display the first row of the weather DataFrame to preview its structure
df_weather_conditions.head(1)


,measurement_date,daily_total_precipitation_mm,daily_maximum_temperature_c,daily_minimum_temperature_c,daily_average_wind_speed_mps,daily_average_temperature_c,temperature_classification,precipitation_classification,wind_classification,weather_severity
0,2022-12-09,0.0,32.3,17.4,2.4,24.85,Warm,No precipitation,Gentle to Fresh Breeze,Moderate


In [27]:
# Preview the first row of the products DataFrame to check column names and initial data
df_products.head(1)

,product,product_id,category,sub_category,shelf_life_days,maximum_days_on_sale,min_stock,max_stock,seasonality,storage_recommendation,unit_of_measurement,barcode_ean,reorder_point,supplier,supplier_id,distance_km,supplier_rating
0,Strawberries,1678467|P,Fresh Foods,Fruits,4,2,10,25,"[July, August, September, October, November]",Refrigerated,lb,8712345000018,10,FreshHarvest Ltd.,1875356|S,84,5


In [28]:
# Rename the column 'measurement_date' to 'date' for easier reference.
df_weather_conditions.rename(columns={'measurement_date': 'received_date'}, inplace=True)


In [29]:
# Filters df_weather_conditions to retain only relevant weather-related columns for analysis or merging
df_weather_conditions = df_weather_conditions[['received_date', 'temperature_classification', 
                                               'precipitation_classification', 'wind_classification', 
                                               'weather_severity']]

In [30]:
# Display the first few rows of the weather DataFrame
df_weather_conditions.head()

,received_date,temperature_classification,precipitation_classification,wind_classification,weather_severity
0,2022-12-09,Warm,No precipitation,Gentle to Fresh Breeze,Moderate
1,2022-12-10,Warm,No precipitation,Gentle to Fresh Breeze,Moderate
2,2022-12-11,Warm,No precipitation,Gentle to Fresh Breeze,Moderate
3,2022-12-12,Warm,No precipitation,Gentle to Fresh Breeze,Moderate
4,2022-12-13,Mild to Temperate,Moderate Rain,Gentle to Fresh Breeze,Moderate


In [31]:
# Display the first few rows of the products DataFrame
df_products.head()

,product,product_id,category,sub_category,shelf_life_days,maximum_days_on_sale,min_stock,max_stock,seasonality,storage_recommendation,unit_of_measurement,barcode_ean,reorder_point,supplier,supplier_id,distance_km,supplier_rating
0,Strawberries,1678467|P,Fresh Foods,Fruits,4,2,10,25,"[July, August, September, October, November]",Refrigerated,lb,8712345000018,10,FreshHarvest Ltd.,1875356|S,84,5
1,Strawberries,1678467|P,Fresh Foods,Fruits,4,2,10,25,"[July, August, September, October, November]",Refrigerated,lb,8712345000018,10,PrimeProduce,1705147|S,238,3
2,Strawberries,1678467|P,Fresh Foods,Fruits,4,2,10,25,"[July, August, September, October, November]",Refrigerated,lb,8712345000018,10,AgroPrime Foods,1240056|S,101,5
3,Spinach,1458488|P,Fresh Foods,Leafy Greens,7,4,10,25,[],Refrigerated,bunch,8712345000025,8,GreenFields Co.,1285354|S,127,1
4,Spinach,1458488|P,Fresh Foods,Leafy Greens,7,4,10,25,[],Refrigerated,bunch,8712345000025,8,UrbanFarmers,1068611|S,95,3


In [32]:
# Determine the number of samples based on the length of the supply DataFrame
n_samples = len(df_weather_conditions)

# Randomly select 'n_samples' rows from the df_products DataFrame
# Sampling is done with replacement (same row can be chosen more than once)
# The index is reset to avoid keeping the original row indices
random_samples1 = df_products.sample(n=n_samples, replace=True).reset_index(drop=True)
random_samples2 = df_products.sample(n=n_samples, replace=True).reset_index(drop=True)
random_samples3 = df_products.sample(n=n_samples, replace=True).reset_index(drop=True)
random_samples4 = df_products.sample(n=n_samples, replace=True).reset_index(drop=True)
random_samples5 = df_products.sample(n=n_samples, replace=True).reset_index(drop=True)


In [33]:
# Merge the supply_df and random_samples DataFrames using their index values
# This aligns rows from both DataFrames based on their position (index)
df_merged1 = df_weather_conditions.merge(random_samples1, left_index=True, right_index=True)
df_merged2 = df_weather_conditions.merge(random_samples2, left_index=True, right_index=True)
df_merged3 = df_weather_conditions.merge(random_samples3, left_index=True, right_index=True)
df_merged4 = df_weather_conditions.merge(random_samples4, left_index=True, right_index=True)
df_merged5 = df_weather_conditions.merge(random_samples5, left_index=True, right_index=True)


In [34]:
# Combine all merged DataFrames into one by stacking them row-wise and resetting the index
df_merged_full = pd.concat([df_merged1, df_merged2, df_merged3, df_merged4, df_merged5], axis=0, ignore_index=True)


In [35]:
# Display DataFrame information
df_merged_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   received_date                 4960 non-null   datetime64[ns]
 1   temperature_classification    4960 non-null   object        
 2   precipitation_classification  4960 non-null   object        
 3   wind_classification           4960 non-null   object        
 4   weather_severity              4960 non-null   object        
 5   product                       4960 non-null   object        
 6   product_id                    4960 non-null   object        
 7   category                      4960 non-null   object        
 8   sub_category                  4960 non-null   object        
 9   shelf_life_days               4960 non-null   object        
 10  maximum_days_on_sale          4960 non-null   object        
 11  min_stock                     

In [36]:
df_merged_full['supplier_rating'].nunique()

5

In [37]:
# Randomly selects 3,000 rows from df_merged_full and resets the index to avoid retaining the original indices
df_raw = df_merged_full.sample(3000).reset_index(drop=True)

In [38]:
df_raw

,received_date,temperature_classification,precipitation_classification,wind_classification,weather_severity,product,product_id,category,sub_category,shelf_life_days,...,max_stock,seasonality,storage_recommendation,unit_of_measurement,barcode_ean,reorder_point,supplier,supplier_id,distance_km,supplier_rating
0,2023-11-02,Mild to Temperate,Moderate Rain,Moderate to Strong Wind,Moderate,Kale,1278267|P,Fresh Foods,Leafy Greens,6,...,25,[],Refrigerated,bunch,8712345000186,8,EcoGrocers,1638538|S,312,5
1,2025-02-14,Warm,Moderate Rain,Gentle to Fresh Breeze,Moderate,Lemon,1141797|P,Fresh Foods,Fruits,21,...,25,[],Refrigerated,lb,8712345000629,9,BioSupply,1569515|S,421,3
2,2024-05-29,Cool,Light Rain,Gentle to Fresh Breeze,Moderate,Apricot,1408388|P,Fresh Foods,Fruits,4,...,25,"[November, December, January]","Room temperature until ripe, then refrigerated",lb,8712345000148,6,PureHarvest,1217776|S,174,5
3,2024-01-28,Mild to Temperate,No precipitation,Gentle to Fresh Breeze,Moderate,Digestive Biscuit,1949506|P,Breads & Biscuits,Biscuits,90,...,25,[],"Cool, dry place in an airtight container",box,8712345001152,10,Sunrise Traders,1476246|S,1890,5
4,2025-04-17,Mild to Temperate,No precipitation,Calm / Light Breeze,Normal,Mushrooms,1954947|P,Fresh Foods,Vegetables,4,...,25,[],"Refrigerated, in a paper bag",pack,8712345000032,7,SupplyExpress Delivery,1023279|S,119,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2023-01-19,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Carrot,1893836|P,Fresh Foods,Vegetables,21,...,25,[],Refrigerated,lb,8712345000599,9,FreshHarvest Ltd.,1875356|S,84,5
2996,2025-03-30,Warm,No precipitation,Gentle to Fresh Breeze,Moderate,Milk,1789695|P,Dairy,Milk,7,...,25,[],Refrigerated,carton,8712345000483,15,SupplyEasy Logistics,1769087|S,654,1
2997,2025-01-02,Warm,Heavy Rain,Gentle to Fresh Breeze,Severe,Cucumber,1541960|P,Fresh Foods,Vegetables,5,...,25,[],Refrigerated,lb,8712345000049,10,AgroNova,1199134|S,105,5
2998,2024-08-05,Mild to Temperate,Moderate Rain,Gentle to Fresh Breeze,Moderate,Ricotta Cheese,1282997|P,Dairy,Cheeses,14,...,25,[],Refrigerated,tub,8712345000223,8,SupplyWorld Logistics,1614374|S,621,1


In [39]:
# Defines the desired column order for organizing the dataset, prioritizing product, inventory, and weather-related attributes
reorder_columns = ['received_date', 'seasonality', 'product', 'product_id', 'category', 'sub_category', 
                   'shelf_life_days', 'maximum_days_on_sale', 'min_stock', 'max_stock', 'reorder_point', 
                   'unit_of_measurement', 'barcode_ean', 'supplier_rating', 'supplier', 'supplier_id', 'distance_km', 
                   'storage_recommendation', 'temperature_classification', 'precipitation_classification', 'wind_classification', 'weather_severity']

# Reorders columns according to reorder_columns
df_raw = df_raw[reorder_columns]

In [40]:
# Converts selected inventory-related columns to integer type for numerical operations and consistency
cols_int = ['shelf_life_days', 'min_stock', 'max_stock', 'reorder_point']
df_raw[cols_int] = df_raw[cols_int].astype(int)

# Converts selected categorical columns to 'category' dtype to optimize memory and improve model performance
cols_cat = ['category', 'sub_category', 'unit_of_measurement', 'supplier_rating', 
            'temperature_classification', 'precipitation_classification', 
            'wind_classification', 'weather_severity']
df_raw[cols_cat] = df_raw[cols_cat].astype('category')


In [41]:
# Summary 
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   received_date                 3000 non-null   datetime64[ns]
 1   seasonality                   3000 non-null   object        
 2   product                       3000 non-null   object        
 3   product_id                    3000 non-null   object        
 4   category                      3000 non-null   category      
 5   sub_category                  3000 non-null   category      
 6   shelf_life_days               3000 non-null   int64         
 7   maximum_days_on_sale          3000 non-null   object        
 8   min_stock                     3000 non-null   int64         
 9   max_stock                     3000 non-null   int64         
 10  reorder_point                 3000 non-null   int64         
 11  unit_of_measurement           

## Generate data about holidays, weekdays, and seasons of the year.

In [42]:
# Creates a full copy of df_raw and assigns it to df_date for independent manipulation
df_date = df_raw.copy()

In [43]:
# Apply the 'day_classification' function from 'create_data_functions' to each date in the 'date' column,
# assigning the result to a new column called 'day_classification'.
df_date['day_classification'] = create_data_functions.day_classification(dates = df_date['received_date'], country='BR')

# Create a boolean column indicating whether the day is classified as a holiday.
df_date['is_holiday'] = np.where(df_date['day_classification'] == 'Holiday', True, False)

# Create a boolean column indicating whether the day falls on a weekend (Saturday or Sunday).
df_date['is_weekend'] = np.where(df_date['received_date'].dt.dayofweek > 4, True, False)

In [44]:
def check_seasonality(row):
    """
    Checks whether the received month of a product aligns with its seasonal availability.
    """
    received_month = row['month_name']
    seasonality_list = row['seasonality']
    
    return received_month in seasonality_list


In [45]:
# Extracts the full month name from 'received_date' to support seasonality checks
df_date['month_name'] = df_date['received_date'].dt.month_name()

# Inserts a new column 'in_season' at position 2, indicating whether the product's received month aligns with its seasonal availability
df_date.insert(2, 'in_season', df_date.apply(check_seasonality, axis=1))

# Removes the temporary 'month_name' column after seasonality classification is complete
df_date.drop(columns=['month_name', 'seasonality'], inplace=True)

## Generate data for stock quantities and sales volumes.

In [46]:
# Creates a full copy
df_stock = df_date.copy()

In [47]:
# Adds a 'sales_demand' column to df_stock by classifying each date using Brazilian holiday calendar and demand heuristics
df_stock['sales_demand'] = create_data_functions.classify_grocery_demand(
    dates=df_stock['received_date'],
    country='BR'
)

In [48]:
df_stock.head(1)

,received_date,in_season,product,product_id,category,sub_category,shelf_life_days,maximum_days_on_sale,min_stock,max_stock,...,distance_km,storage_recommendation,temperature_classification,precipitation_classification,wind_classification,weather_severity,day_classification,is_holiday,is_weekend,sales_demand
0,2023-11-02,False,Kale,1278267|P,Fresh Foods,Leafy Greens,6,4,10,25,...,312,Refrigerated,Mild to Temperate,Moderate Rain,Moderate to Strong Wind,Moderate,Holiday,True,False,Very High)


In [49]:
# Inserts a new column 'stock_qty' at position 7 with simulated stock quantities generated from min and max stock values
df_stock.insert(8, 'stock_qty', create_data_functions.create_stock_distribution_vectorized(df_stock['min_stock'], df_stock['max_stock']))

In [50]:
# Insert a new column 'sales_volume' at position 8 with initial value 0
df_stock.insert(9, 'sales_volume', 0)

# Replace the 'sales_volume' column with simulated values using a custom function
df_stock['sales_volume'] = create_data_functions.simulate_sales_volume(df_stock, random_state=42)

In [52]:
# Insert a new column 'lpo' at position 1 with initial value 0
df_stock.insert(1, 'lpo', 0)

In [53]:
# Generate simulated purchase order dates based on product attributes and logistics
df_stock['lpo'] = create_data_functions.simulate_purchase_order_columns(df_stock)

In [54]:
# Save the updated DataFrame to CSV, excluding the index column
df_stock.to_csv(raw_data_path + 'synthetic_data_grocery_stock.csv', index=False)